In [1]:
import numpy as np
from astropy import units as u
from astropy import time

from poliastro.twobody.propagation import propagate
from poliastro.twobody.propagation import cowell
from poliastro.bodies import Earth
from poliastro.plotting import OrbitPlotter3D

from perylune import geom, orbit_tools, perturbers

# Needed to handle TLE into Poliastro's Orbit
from tletools import TLE

In [2]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [3]:
# Let's get the initial PW-Sat2 orbit. Its norad id is 43814
# PW-SAT2 TLE data from 2020-03-15, as published by Celestrak
tle_text = """PW-SAT2                 
1 43814U 18099BJ  20049.76768004  .00014282  00000-0  95610-3 0  9991
2 43814  97.7801 128.6236 0010365 203.2339 156.8317 15.06987254 65660"""
tle_lines = tle_text.strip().splitlines()
tle_pwsat2 = TLE.from_lines(*tle_lines)
orb1 = tle_pwsat2.to_orbit()

orbit_tools.print_orb(orb1)

6916 x 6931 km x 97.8 deg (GCRS) orbit around Earth (♁) at epoch 2020-02-18T18:25:27.555456000 (UTC)
a(𝑎)=6923.5493km, b=6923.5456km, e=0.00, i=97.78deg raan(Ω)=128.62deg argp(𝜔)=203.23deg nu(𝜈)=156.88deg
period=5733.29s perapis=6916.3730km(538.24km) apoapsis=6930.7256km(552.59km)


In [8]:
# Calculate propagation for the 10 orbits and sample 1000 points along the way
# Interesting values: 2e-5 - spiral, 2e-4 - fast escape trajectory
times = np.linspace(0, 10 * orb1.period, 1000)
positions = propagate(orb1, time.TimeDelta(times), method=cowell, rtol=1e-11, ad=perturbers.constant_accel(2e-5) )

In [9]:
from plotly.graph_objects import Figure, Layout, Scatter, Scatter3d

frame = OrbitPlotter3D()
frame.set_attractor(Earth)

frame.plot(orb1, label="Initial orbit")
frame.plot_trajectory(positions, color="green", label="PW-Sat2 (modified)")


#frame._draw_point(696*u.km, "red", "X", center=[14700, 0, 0] * u.km)
#frame._draw_point(696*u.km, "yellow", "Y", center=[0, 14700, 0] * u.km)
#frame._draw_point(696*u.km, "green", "Z", center=[0, 0, 14700] * u.km)

frame._layout.width = 1200
frame._layout.height = 800
frame._layout.margin=dict(l=10, r=10, b=10, t=10, pad=4 )
frame.show()